# de base de datos a json

In [3]:

from fastapi import FastAPI, requests, HTTPException
import uvicorn
import os
from dotenv import load_dotenv
import psycopg2
import json
import pandas as pd
load_dotenv()
    
#Leer Las Variables:
host = os.getenv("DB_HOST_AWS")
username = os.getenv("DB_USER_AWS")
password = os.getenv("DB_PASSWORD_AWS")
database = os.getenv("DB_DATABASE_AWS")
port = os.getenv("DB_PORT_AWS")
try:
    conn = psycopg2.connect(host=host,
                                database=database,
                                user=username,
                                password=password,
                                port=port,
                                sslmode="require")
    
    print("Conexión exitosa a la base de datos PostgreSQL con SSL")
except psycopg2.OperationalError as e:
    raise HTTPException(status_code=400, detail="Error de conexión: " + str(e))
except Exception as error:
    raise HTTPException(status_code=400, detail="Error desconocido: " + str(e))
cur = conn.cursor()


Conexión exitosa a la base de datos PostgreSQL con SSL


In [31]:
# Escribe la consulta SQL
query = """SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public' AND table_type = 'BASE TABLE';
"""
# query = "SELECT * FROM preguntas_front"

# Usa pandas para ejecutar la consulta y convertirla en un DataFrame
df = pd.read_sql_query(query, conn)
df.head(50)

C:\Users\2012f\AppData\Local\Temp\ipykernel_17420\4123380349.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,table_name
0,no_sociosanit_formulario
1,opciones_chatbot
2,respuestas_modelo
3,admin_data
4,preguntas_opciones_chatbot
5,categorias_chatbot
6,preguntas
7,respuestas_chatbot_tengo_vih
8,respuestas_chatbot_exposicion_vih
9,respuestas_chatbot_informacion_vih


In [ ]:
# Escribe la consulta SQL
id_usuario = '1234abcd'
query = """
        DELETE FROM admin_data
        WHERE admin_id = 1;
        """
cur.execute(query, (id_usuario,))  # Usar parámetros para evitar inyección SQL

# Confirmar los cambios
conn.commit()

# # Escribe la consulta SQL
# query = """
#         UPDATE admin_data 
# SET password = '$2b$12$EjemploDeContraseñaHasheadaQwerty'
# WHERE admin_id = 1;

#         """
# cur.execute(query)  # Usar parámetros para evitar inyección SQL

# # Confirmar los cambios
# conn.commit()

        


In [ ]:
import bcrypt

def check_admin_details(email: str, password: str) -> bool:
    # conn = connect_to_db()  
    if not conn:
        raise RuntimeError("Database connection could not be established.")
    
    try:
        with conn.cursor() as cur:
            # Consulta para obtener el hash almacenado para el email
            query = "SELECT password FROM admin_data WHERE email = %s;"
            cur.execute(query, (email,))
            result = cur.fetchone()
            
            if result:
                # Obtén el hash almacenado
                stored_hash = result[0]
                
                # Compara la contraseña ingresada con el hash almacenado
                if bcrypt.checkpw(password.encode('utf-8'), stored_hash.encode('utf-8')):
                    return True  # Credenciales válidas
                else:
                    return False  # Credenciales inválidas
            else:
                return False  # Email no encontrado
        
    except Exception as e:
        raise RuntimeError(f"Error fetching data: {e}")
    finally:
        conn.close()

In [6]:
import bcrypt

def add_admin(email: str, plain_password: str):
    try:
        # Hashea la contraseña
        hashed_password = bcrypt.hashpw(plain_password.encode('utf-8'), bcrypt.gensalt()).decode('utf-8')
        
        # Inserta el email y la contraseña hasheada en la base de datos
        query = "INSERT INTO admin_data (email, password) VALUES (%s, %s);"
        cur.execute(query, (email, hashed_password))
        
        # Confirma los cambios en la base de datos
        conn.commit()
        print("Admin añadido con éxito.")
    except Exception as e:
        print(f"Error al añadir admin: {e}")
    finally:
        if cur:
            cur.close()

# Ejemplo de uso
add_admin("2oadmin@gmail.com", "hansJuapo")


Admin añadido con éxito.


In [7]:
# Escribe la consulta SQL
query = """
        SELECT * FROM admin_data
        """
df = pd.read_sql_query(query, conn)
df.head(50)

C:\Users\2012f\AppData\Local\Temp\ipykernel_10052\2327992602.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,admin_id,email,password
0,1,felgtbiqplus@gmail.com,$2b$12$EjemploDeContraseñaHasheadaQwerty
1,2,2oadmin@gmail.com,$2b$12$qcXFvRhe0XhlBn4A4BYs0.Oy8VreGotRUDDdEbp...


In [ ]:
# Escribe la consulta SQL
query = """
        SELECT id_categoria,titulo_categoria FROM categorias_chatbot
        WHERE seccion = 'sociosanitario'
        """
df = pd.read_sql_query(query, conn)
df.head(50)

In [11]:
import pandas as pd

# Convertir a JSON clave-valor
json_result = df.set_index("id_categoria")["titulo_categoria"].to_dict()

# Imprimir el JSON
import json
print(json.dumps(json_result, indent=2, ensure_ascii=False))


{
  "6": "Tengo vih",
  "7": "Creo que me he expuesto al virus",
  "8": "Quiero saber mas sobre el vih/sida",
  "9": "Estoy apoyando a una persona seropositiva"
}


# generar el JSON FRONT 

## primer nivel Categorias

In [7]:
# Escribe la consulta SQL
query = """
        SELECT id_categoria,titulo_categoria FROM categorias_chatbot
        WHERE seccion = 'usuario'
        """
df_categorias = pd.read_sql_query(query, conn)



C:\Users\2012f\AppData\Local\Temp\ipykernel_20404\3613681017.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_categorias = pd.read_sql_query(query, conn)


In [9]:
import pandas as pd

# Crear la estructura base del JSON
data = [
    { "id": "1", "message": "Elige un tema:", "trigger": "2" },
    {
        "id": "2",
        "options": []
    }
]

# Agregar las opciones dinámicamente
for idx, row in df.iterrows():
    option = {
        "value": f"{row['titulo_categoria']}",
        "label": row['titulo_categoria'],
        "trigger": str(int(row['id_categoria']) + 1)  # Establece el trigger como el siguiente número
    }
    data[1]["options"].append(option)

# Imprimir el resultado final
import json
print(json.dumps(data, indent=2, ensure_ascii=False))


[
  {
    "id": "1",
    "message": "Elige un tema:",
    "trigger": "2"
  },
  {
    "id": "2",
    "options": [
      {
        "value": "Personal Sanitario",
        "label": "Personal Sanitario",
        "trigger": "2"
      },
      {
        "value": "Trabajador Social",
        "label": "Trabajador Social",
        "trigger": "3"
      },
      {
        "value": "Psicologo",
        "label": "Psicologo",
        "trigger": "4"
      },
      {
        "value": "Educador",
        "label": "Educador",
        "trigger": "5"
      },
      {
        "value": "Voluntarios y Cuidadores",
        "label": "Voluntarios y Cuidadores",
        "trigger": "6"
      }
    ]
  }
]


## Segundo nivel prguntas

In [20]:
# Escribe la consulta SQL
query = """
        SELECT 
    categoria_pregunta_chat_intermed.id_categoria,
    categoria_pregunta_chat_intermed.id_pregunta,
    preguntas_chatbot.texto_pregunta
FROM 
    categoria_pregunta_chat_intermed
JOIN 
    preguntas_chatbot
ON 
    categoria_pregunta_chat_intermed.id_pregunta = preguntas_chatbot.id_pregunta;

        """
df_preguntas = pd.read_sql_query(query, conn)



C:\Users\2012f\AppData\Local\Temp\ipykernel_16992\2413182831.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_preguntas = pd.read_sql_query(query, conn)


In [27]:
# Crear la estructura inicial del JSON
data = [
    { "id": "1", "message": "Elige un tema:", "trigger": "2" },
    {
        "id": "2",
        "options": []
    }
]

# Agregar las categorías como opciones del primer nivel
for idx, row in df_categorias.iterrows():
    option = {
        "value": row['titulo_categoria'],
        "label": row['titulo_categoria'],
        "trigger": str(row['id_categoria'])  # Trigger que apunta al id_categoria
    }
    data[1]["options"].append(option)

# Generar el segundo nivel (preguntas asociadas a cada categoría)
categorias = df_preguntas.groupby("id_categoria")
for id_categoria, group in categorias:
    step = {
        "id": str(id_categoria),  # El id del paso corresponde al id_categoria
        "message": "Elige una pregunta:",
        "options": []
    }
    for _, question in group.iterrows():
        option = {
            "value": question['texto_pregunta'],
            "label": question['texto_pregunta'],
            "trigger": "end"  # Aquí termina el flujo, ajustar según necesidad
        }
        step["options"].append(option)
    data.append(step)

# Imprimir el JSON final
import json
print(json.dumps(data, indent=2, ensure_ascii=False))


[
  {
    "id": "1",
    "message": "Elige un tema:",
    "trigger": "2"
  },
  {
    "id": "2",
    "options": [
      {
        "value": "Tengo vih",
        "label": "Tengo vih",
        "trigger": "6"
      },
      {
        "value": "Creo que me he expuesto al virus",
        "label": "Creo que me he expuesto al virus",
        "trigger": "7"
      },
      {
        "value": "Quiero saber mas sobre el vih/sida",
        "label": "Quiero saber mas sobre el vih/sida",
        "trigger": "8"
      },
      {
        "value": "Estoy apoyando a una persona seropositiva",
        "label": "Estoy apoyando a una persona seropositiva",
        "trigger": "9"
      }
    ]
  },
  {
    "id": "1",
    "message": "Elige una pregunta:",
    "options": [
      {
        "value": "¿Qué necesitas como Personal Sanitario?",
        "label": "¿Qué necesitas como Personal Sanitario?",
        "trigger": "end"
      },
      {
        "value": "¿Quieres información sobre algun tema?",
        "labe

# Cargar datos

In [264]:
# Recuperar los id_pregunta de la tabla preguntas_chatbot
cur.execute("SELECT id_pregunta, texto_pregunta FROM preguntas_chatbot")
preguntas_dict = cur.fetchall()  # Esto es una lista de tuplas con (id_pregunta, pregunta)

# Recuperar los id_opcion de la tabla opciones_chatbot
cur.execute("SELECT id_opcion, texto_opcion FROM opciones_chatbot")
opciones_dict = cur.fetchall()  # Esto es una lista de tuplas con (id_opcion, opcion)

# Crear mapeo de preguntas a ids
preguntas_dict = {pregunta[1]: pregunta[0] for pregunta in preguntas_dict}

# Crear mapeo de opciones a ids
opciones_dict = {opcion[1]: opcion[0] for opcion in opciones_dict}

print(preguntas_dict)
print(opciones_dict)



{'¿Qué apoyo necesitas?': 1, '¿Qué necesitas como Trabajador Social?': 2, '¿Cuándo ocurrió la posible infección?': 3, '¿Necesitas recursos de referencia?': 4, '¿Tienes acceso a un médico?': 5, '¿Has compartido tu preocupación con alguien?': 6, '¿Qué necesitas como Psicologo?': 7, '¿Ha sido en un entorno de "chem-sex"?': 8, '¿Estás en tratamiento TAR?': 9, '¿Has compartido tu diagnostico con alguien?': 10, '¿Tiene acceso a recursos locales o grupos de apoyo?': 11, '¿Qué necesitas como Voluntario/Cuidador?': 12, '¿Cuándo te diagnosticaron?': 13, '¿Has compartido su preocupación sobre esta persona con alguien?': 14, '¿Sabes qué es la PEP?': 15, '¿Qué necesitas como Personal Sanitario?': 16, '¿Qué tipo de exposición fue?': 17, '¿Qué necesitas como Educador?': 18, '¿Quieres información sobre algun tema?': 19}
{'Manejo clínico de pacientes con vih': 1, 'Protocolo PEP': 2, 'Tratamientos (PREP, TAR)': 3, 'Prevención de infecciones oportunistas': 4, 'Consejería para adherencia al tratamiento': 

In [265]:
# Lista de relaciones entre preguntas y opciones (basado en los valores que tienes)
# Lista de relaciones entre preguntas y opciones con los IDs correctos
preguntas_opciones = [
    (preguntas_dict['¿Qué necesitas como Personal Sanitario?'], opciones_dict['Manejo clínico de pacientes con vih']),
    (preguntas_dict['¿Qué necesitas como Personal Sanitario?'], opciones_dict['Protocolo PEP']),
    (preguntas_dict['¿Qué necesitas como Personal Sanitario?'], opciones_dict['Tratamientos (PREP, TAR)']),
    (preguntas_dict['¿Qué necesitas como Personal Sanitario?'], opciones_dict['Prevención de infecciones oportunistas']),
    (preguntas_dict['¿Qué necesitas como Personal Sanitario?'], opciones_dict['Consejería para adherencia al tratamiento']),

    (preguntas_dict['¿Qué necesitas como Trabajador Social?'], opciones_dict['Acceso a medicamentos o servicios']),
    (preguntas_dict['¿Qué necesitas como Trabajador Social?'], opciones_dict['Recursos legales y derechos']),
    (preguntas_dict['¿Qué necesitas como Trabajador Social?'], opciones_dict['Apoyo a personas en situación de vulnerabilidad']),
    (preguntas_dict['¿Qué necesitas como Trabajador Social?'], opciones_dict['Conexión con grupos de apoyo comunitario']),
    (preguntas_dict['¿Qué necesitas como Trabajador Social?'], opciones_dict['Información sobre redes de Servicios Sociales']),

    (preguntas_dict['¿Qué necesitas como Psicologo?'], opciones_dict['Apoyo emocional para personas recién diagnosticadas']),
    (preguntas_dict['¿Qué necesitas como Psicologo?'], opciones_dict['Intervenciones para adherencia al tratamiento']),
    (preguntas_dict['¿Qué necesitas como Psicologo?'], opciones_dict['Manejo del estigma y problemas de salud mental']),
    (preguntas_dict['¿Qué necesitas como Psicologo?'], opciones_dict['Recursos para pacientes con vih y trastornos psicológicos']),
    (preguntas_dict['¿Qué necesitas como Psicologo?'], opciones_dict['Consejería en prevención y autocuidado']),

    (preguntas_dict['¿Qué necesitas como Educador?'], opciones_dict['Material educativo sobre vih']),
    (preguntas_dict['¿Qué necesitas como Educador?'], opciones_dict['Capacitación en prevención']),
    (preguntas_dict['¿Qué necesitas como Educador?'], opciones_dict['Métodos para combatir el estigma']),
    (preguntas_dict['¿Qué necesitas como Educador?'], opciones_dict['Recursos para sensibilización']),
    (preguntas_dict['¿Qué necesitas como Educador?'], opciones_dict['Estadísticas y datos actualizados']),

    (preguntas_dict['¿Qué necesitas como Voluntario/Cuidador?'], opciones_dict['Info básica sobre vih']),
    (preguntas_dict['¿Qué necesitas como Voluntario/Cuidador?'], opciones_dict['Consejos para apoyar emocionalmente']),
    (preguntas_dict['¿Qué necesitas como Voluntario/Cuidador?'], opciones_dict['Recursos legales y sociales para pacientes']),
    (preguntas_dict['¿Qué necesitas como Voluntario/Cuidador?'], opciones_dict['Conexión con grupos de apoyo comunitario']),
    (preguntas_dict['¿Qué necesitas como Voluntario/Cuidador?'], opciones_dict['Métodos de autocuidado para cuidadores']),
    
    (preguntas_dict['¿Cuándo te diagnosticaron?'], opciones_dict['Hace menos de 6 meses']),
    (preguntas_dict['¿Cuándo te diagnosticaron?'], opciones_dict['Entre 6 meses y un año']),
    (preguntas_dict['¿Cuándo te diagnosticaron?'], opciones_dict['Hace más de un año']),
    
    (preguntas_dict['¿Estás en tratamiento TAR?'], opciones_dict['Sí']),
    (preguntas_dict['¿Estás en tratamiento TAR?'], opciones_dict['No']),
    (preguntas_dict['¿Estás en tratamiento TAR?'], opciones_dict['No estoy seguro']),

    (preguntas_dict['¿Tienes acceso a un médico?'], opciones_dict['Sí']),
    (preguntas_dict['¿Tienes acceso a un médico?'], opciones_dict['No']),

    (preguntas_dict['¿Has compartido tu diagnostico con alguien?'], opciones_dict['Un amigo']),
    (preguntas_dict['¿Has compartido tu diagnostico con alguien?'], opciones_dict['Algún familiar']),
    (preguntas_dict['¿Has compartido tu diagnostico con alguien?'], opciones_dict['Mi pareja en ese momento']),
    (preguntas_dict['¿Has compartido tu diagnostico con alguien?'], opciones_dict['Compañero/a de trabajo']),
    (preguntas_dict['¿Has compartido tu diagnostico con alguien?'], opciones_dict['Con mi jefe/a']),
    (preguntas_dict['¿Has compartido tu diagnostico con alguien?'], opciones_dict['Personal de ONG']),
    (preguntas_dict['¿Has compartido tu diagnostico con alguien?'], opciones_dict['Expareja']),
    (preguntas_dict['¿Has compartido tu diagnostico con alguien?'], opciones_dict['Nadie']),

    (preguntas_dict['¿Quieres información sobre algun tema?'], opciones_dict['Opciones de tratamiento']),
    (preguntas_dict['¿Quieres información sobre algun tema?'], opciones_dict['Apoyo psicológico']),
    (preguntas_dict['¿Quieres información sobre algun tema?'], opciones_dict['Derechos laborales y legales']),
    (preguntas_dict['¿Quieres información sobre algun tema?'], opciones_dict['Grupos de apoyo']),
    (preguntas_dict['¿Quieres información sobre algun tema?'], opciones_dict['Prevención de transmisión']),

    (preguntas_dict['¿Cuándo ocurrió la posible infección?'], opciones_dict['Últimas 72h']),
    (preguntas_dict['¿Cuándo ocurrió la posible infección?'], opciones_dict['Hace más de 72h']),

    (preguntas_dict['¿Qué tipo de exposición fue?'], opciones_dict['Relación sexual']),
    (preguntas_dict['¿Qué tipo de exposición fue?'], opciones_dict['Aguja compartida']),
    (preguntas_dict['¿Qué tipo de exposición fue?'], opciones_dict['Contacto con fluidos corporales (sangre, fluido sexual..)']),
    (preguntas_dict['¿Qué tipo de exposición fue?'], opciones_dict['No estoy seguro']),

    (preguntas_dict['¿Ha sido en un entorno de "chem-sex"?'], opciones_dict['Sí']),
    (preguntas_dict['¿Ha sido en un entorno de "chem-sex"?'], opciones_dict['No']),

    (preguntas_dict['¿Tienes acceso a un médico?'], opciones_dict['Sí']),
    (preguntas_dict['¿Tienes acceso a un médico?'], opciones_dict['No']),

    (preguntas_dict['¿Has compartido tu preocupación con alguien?'], opciones_dict['La persona que me preocupa']),
    (preguntas_dict['¿Has compartido tu preocupación con alguien?'], opciones_dict['Un amigo']),
    (preguntas_dict['¿Has compartido tu preocupación con alguien?'], opciones_dict['Algún familiar']),
    (preguntas_dict['¿Has compartido tu preocupación con alguien?'], opciones_dict['Mi pareja en ese momento']),
    (preguntas_dict['¿Has compartido tu preocupación con alguien?'], opciones_dict['Compañero/a de trabajo']),
    (preguntas_dict['¿Has compartido tu preocupación con alguien?'], opciones_dict['Con mi jefe/a']),
    (preguntas_dict['¿Has compartido tu preocupación con alguien?'], opciones_dict['Personal de ONG']),
    (preguntas_dict['¿Has compartido tu preocupación con alguien?'], opciones_dict['Expareja']),
    (preguntas_dict['¿Has compartido tu preocupación con alguien?'], opciones_dict['Nadie']),

    (preguntas_dict['¿Sabes qué es la PEP?'], opciones_dict['Si, quiero más información']),
    (preguntas_dict['¿Sabes qué es la PEP?'], opciones_dict['No ¿Qué es?']),

    (preguntas_dict['¿Necesitas recursos de referencia?'], opciones_dict['¿Qué es el vih/sida?']),
    (preguntas_dict['¿Necesitas recursos de referencia?'], opciones_dict['Formas de transmisión']),
    (preguntas_dict['¿Necesitas recursos de referencia?'], opciones_dict['Métodos de prevención']),
    (preguntas_dict['¿Necesitas recursos de referencia?'], opciones_dict['Impacto del tratamiento']),
    (preguntas_dict['¿Necesitas recursos de referencia?'], opciones_dict['Historia del vih']),

    (preguntas_dict['¿Tiene acceso a recursos locales o grupos de apoyo?'], opciones_dict['Sí']),
    (preguntas_dict['¿Tiene acceso a recursos locales o grupos de apoyo?'], opciones_dict['No']),

    (preguntas_dict['¿Has compartido su preocupación sobre esta persona con alguien?'], opciones_dict['La persona que me preocupa']),
    (preguntas_dict['¿Has compartido su preocupación sobre esta persona con alguien?'], opciones_dict['Un amigo']),
    (preguntas_dict['¿Has compartido su preocupación sobre esta persona con alguien?'], opciones_dict['Algún familiar']),
    (preguntas_dict['¿Has compartido su preocupación sobre esta persona con alguien?'], opciones_dict['Mi pareja en ese momento']),
    (preguntas_dict['¿Has compartido su preocupación sobre esta persona con alguien?'], opciones_dict['Compañero/a de trabajo']),
    (preguntas_dict['¿Has compartido su preocupación sobre esta persona con alguien?'], opciones_dict['Con mi jefe/a']),
    (preguntas_dict['¿Has compartido su preocupación sobre esta persona con alguien?'], opciones_dict['Personal de ONG']),
    (preguntas_dict['¿Has compartido su preocupación sobre esta persona con alguien?'], opciones_dict['Expareja']),
    (preguntas_dict['¿Has compartido su preocupación sobre esta persona con alguien?'], opciones_dict['Nadie']),

]


for pregunta_id, opcion_id in preguntas_opciones:
    cur.execute("""
        SELECT 1 FROM preguntas_opciones_chatbot
        WHERE id_pregunta = %s AND id_opcion = %s;
    """, (pregunta_id, opcion_id))
    
    # Si no existe la combinación, insertar
    if not cur.fetchone():
        cur.execute("""
            INSERT INTO preguntas_opciones_chatbot (id_pregunta, id_opcion)
            VALUES (%s, %s);
        """, (pregunta_id, opcion_id))

# Confirmar los cambios en la base de datos
conn.commit()


In [280]:
cur = conn.cursor()

# Query para eliminar la tabla
query = """
ALTER TABLE categorias_chatbot
ADD COLUMN seccion VARCHAR(255);
"""

# Ejecutar la eliminación
cur.execute(query)
conn.commit()

# JUNTAR EN UN TABLA CATEGORIAS Y OPCIONES

## Usuarios

In [6]:
# Escribe la consulta SQL
query_usuarios = """
            SELECT 
                c.id_categoria,
                p.id_pregunta,
                o.id_opcion,
                c.titulo_categoria,
                p.texto_pregunta,
                o.texto_opcion
            FROM 
                categorias_chatbot c
            JOIN 
                categoria_pregunta_chat_intermed cp ON c.id_categoria = cp.id_categoria
            JOIN 
                preguntas_chatbot p ON cp.id_pregunta = p.id_pregunta
            JOIN 
                preguntas_opciones_chatbot po ON p.id_pregunta = po.id_pregunta
            JOIN 
                opciones_chatbot o ON po.id_opcion = o.id_opcion
            WHERE 
                c.seccion = 'usuario'
            ORDER BY 
                c.id_categoria, p.id_pregunta, o.id_opcion;
            """


df = pd.read_sql_query(query_usuarios, conn)
df.head(50)




# # Convertir el DataFrame a JSON
# # Opción 1: Convertir a un diccionario JSON estructurado por filas
# json_data = df.to_json(orient='records', indent=4)

# # Mostrar el JSON por consola (opcional)
# print(json_data)




# def generar_dialogo(data):
#     steps = []
#     trigger_count = 2  # Comenzamos con el trigger 2 como se muestra en el ejemplo

#     for categoria in data:
#         for pregunta in categoria["preguntas"]:
#             # Agregar la pregunta y sus opciones
#             steps.append({
#                 "id": str(trigger_count),
#                 "message": pregunta["texto_pregunta"],
#                 "trigger": str(trigger_count + 1),
#             })
#             trigger_count += 1
            
#             opciones = []
#             for opcion in pregunta["opciones"]:
#                 opciones.append({
#                     "value": opcion["texto_opcion"],
#                     "label": opcion["texto_opcion"],
#                     "trigger": str(trigger_count + 1)
#                 })
#             steps.append({
#                 "id": str(trigger_count),
#                 "options": opciones,
#             })
#             trigger_count += 1

#     # Añadir paso final (end) con el mensaje de fin
#     steps.append({
#         "id": str(trigger_count),
#         "message": "Gracias por tu tiempo",
#         "end": True
#     })

#     return steps

# # Generar el diálogo
# dialogo = generar_dialogo(json_data)

# # Mostrar el resultado
# print(json.dumps(dialogo, indent=2, ensure_ascii=False))

C:\Users\2012f\AppData\Local\Temp\ipykernel_21220\1367161454.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_usuarios, conn)


,id_categoria,id_pregunta,id_opcion,titulo_categoria,texto_pregunta,texto_opcion
0,6,5,28,Tengo vih,¿Tienes acceso a un médico?,Sí
1,6,5,28,Tengo vih,¿Tienes acceso a un médico?,Sí
2,6,5,29,Tengo vih,¿Tienes acceso a un médico?,No
3,6,5,29,Tengo vih,¿Tienes acceso a un médico?,No
4,6,9,28,Tengo vih,¿Estás en tratamiento TAR?,Sí
5,6,9,28,Tengo vih,¿Estás en tratamiento TAR?,Sí
6,6,9,29,Tengo vih,¿Estás en tratamiento TAR?,No
7,6,9,29,Tengo vih,¿Estás en tratamiento TAR?,No
8,6,9,30,Tengo vih,¿Estás en tratamiento TAR?,No estoy seguro
9,6,9,30,Tengo vih,¿Estás en tratamiento TAR?,No estoy seguro


In [22]:
# Resultado de la consulta SQL (simulado)
df = pd.read_sql_query(query_usuarios, conn)
# Agrupar las opciones dentro de las preguntas
grouped_questions = (
    df.groupby(['id_categoria', 'titulo_categoria', 'id_pregunta', 'texto_pregunta'])
    .apply(lambda x: x[['id_opcion', 'texto_opcion']].to_dict(orient='records'))
    .reset_index()
    .rename(columns={0: 'opciones'})
)

# Agrupar preguntas dentro de categorías
grouped_categories = (
    grouped_questions.groupby(['id_categoria', 'titulo_categoria'])
    .apply(lambda x: x[['id_pregunta', 'texto_pregunta', 'opciones']].to_dict(orient='records'))
    .reset_index()
    .rename(columns={0: 'preguntas'})
)

# Convertir a JSON estructurado
categories = grouped_categories.to_dict(orient='records')

# Transformar al formato esperado
nuevo_json = []

# Paso inicial para el chatbot
nuevo_json.append({"id": "1", "message": "Elige un tema:", "trigger": "2"})

# Añadir las categorías
categoria_trigger = 2
for categoria in categories:
    categoria_id = str(categoria['id_categoria'])
    categoria_titulo = categoria['titulo_categoria']
    preguntas = categoria['preguntas']

    # Opciones de categorías
    nuevo_json.append({
        "id": categoria_id,
        "message": f"Selecciona una pregunta de la categoría: {categoria_titulo}",
        "options": [
            {"value": f"{categoria_id}.1", "label": categoria_titulo, "trigger": f"{categoria_id}_preguntas"}
        ]
    })

    # Añadir las preguntas de la categoría
    for pregunta in preguntas:
        pregunta_id = str(pregunta['id_pregunta'])
        opciones = pregunta['opciones']

        nuevo_json.append({
            "id": f"{categoria_id}_preguntas",
            "message": pregunta['texto_pregunta'],
            "options": [
                {"value": str(opcion['id_opcion']), "label": opcion['texto_opcion'], "end": True}
                for opcion in opciones
            ]
        })

# Convertir al JSON final
formatted_json = json.dumps(nuevo_json, indent=4, ensure_ascii=False)

# Mostrar el JSON final
print(formatted_json)

[
    {
        "id": "1",
        "message": "Elige un tema:",
        "trigger": "2"
    },
    {
        "id": "6",
        "message": "Selecciona una pregunta de la categoría: Tengo vih",
        "options": [
            {
                "value": "6.1",
                "label": "Tengo vih",
                "trigger": "6_preguntas"
            }
        ]
    },
    {
        "id": "6_preguntas",
        "message": "¿Tienes acceso a un médico?",
        "options": [
            {
                "value": "28",
                "label": "Sí",
                "end": true
            },
            {
                "value": "28",
                "label": "Sí",
                "end": true
            },
            {
                "value": "29",
                "label": "No",
                "end": true
            },
            {
                "value": "29",
                "label": "No",
                "end": true
            }
        ]
    },
    {
        "id": "6_preguntas"

C:\Users\2012f\AppData\Local\Temp\ipykernel_20404\1019567480.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_usuarios, conn)
C:\Users\2012f\AppData\Local\Temp\ipykernel_20404\1019567480.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x[['id_opcion', 'texto_opcion']].to_dict(orient='records'))
C:\Users\2012f\AppData\Local\Temp\ipykernel_20404\1019567480.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of

# preparar

In [18]:
# Escribe la consulta SQL
query_sociosanitarios = """
    SELECT 
        c.id_categoria,
        p.id_pregunta,
        o.id_opcion,
        c.titulo_categoria,
        p.texto_pregunta,
        o.texto_opcion
    FROM 
        categorias_chatbot c
    JOIN 
        categoria_pregunta_chat_intermed cp ON c.id_categoria = cp.id_categoria
    JOIN 
        preguntas_chatbot p ON cp.id_pregunta = p.id_pregunta
    JOIN 
        preguntas_opciones_chatbot po ON p.id_pregunta = po.id_pregunta
    JOIN 
        opciones_chatbot o ON po.id_opcion = o.id_opcion
    WHERE 
        c.seccion = 'sociosanitario'
    ORDER BY 
        c.id_categoria, p.id_pregunta, o.id_opcion;
"""

# Ejecutar la consulta y obtener el DataFrame
df = pd.read_sql_query(query_sociosanitarios, conn)

df

C:\Users\2012f\AppData\Local\Temp\ipykernel_20404\2312306793.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_sociosanitarios, conn)


,id_categoria,id_pregunta,id_opcion,titulo_categoria,texto_pregunta,texto_opcion


In [ ]:
import pandas as pd
import json

# Resultado de la consulta SQL (simulado)
df = pd.read_sql_query(query_usuarios, conn)




In [6]:
# JSON generado a partir de la consulta SQL
import json

# Ejemplo de entrada basado en `json_data` generado
json_result = json.loads(json_data)

# Transformar al formato deseado
nuevo_json = []

# Paso inicial para el chatbot
nuevo_json.append({"id": "1", "message": "Elige un tema:", "trigger": "2"})

# Agregar categorías como opciones principales
categoria_id = 2  # Trigger inicial para las categorías
for categoria in json_result:
    categoria_trigger = str(categoria_id)
    nuevo_json.append({
        "id": "2",
        "options": [
            {
                "value": f"{categoria['id_categoria']}.{categoria_id}",
                "label": categoria['titulo_categoria'],
                "trigger": categoria_trigger
            }
        ]
    })
    
    # Agregar preguntas de cada categoría
    pregunta_trigger_id = categoria_id + 1
    for pregunta in categoria['preguntas']:
        pregunta_trigger = str(pregunta_trigger_id)
        nuevo_json.append({
            "id": categoria_trigger,
            "message": pregunta['texto_pregunta'],
            "trigger": pregunta_trigger
        })

        # Agregar opciones para cada pregunta
        opciones = [
            {
                "value": str(opcion['id_opcion']),
                "label": opcion['texto_opcion'],
                "trigger": f"end_{pregunta_trigger}"
            }
            for opcion in pregunta['opciones']
        ]
        nuevo_json.append({
            "id": pregunta_trigger,
            "options": opciones
        })
        pregunta_trigger_id += 1

    categoria_id = pregunta_trigger_id  # Actualizar ID para la siguiente categoría

# Convertir a JSON
formatted_json = json.dumps(nuevo_json, indent=4, ensure_ascii=False)

# Mostrar el JSON final por consola
print(formatted_json)


[
    {
        "id": "1",
        "message": "Elige un tema:",
        "trigger": "2"
    },
    {
        "id": "2",
        "options": [
            {
                "value": "1.2",
                "label": "Personal Sanitario",
                "trigger": "2"
            }
        ]
    },
    {
        "id": "2",
        "message": "¿Qué necesitas como Personal Sanitario?",
        "trigger": "3"
    },
    {
        "id": "3",
        "options": [
            {
                "value": "1",
                "label": "Manejo clínico de pacientes con vih",
                "trigger": "end_3"
            },
            {
                "value": "2",
                "label": "Protocolo PEP",
                "trigger": "end_3"
            },
            {
                "value": "3",
                "label": "Tratamientos (PREP, TAR)",
                "trigger": "end_3"
            },
            {
                "value": "4",
                "label": "Prevención de infecciones oport

## Sociosanitario


In [ ]:
# Escribe la consulta SQL
query_sociosanitarios = """
    SELECT 
        c.id_categoria,
        p.id_pregunta,
        o.id_opcion,
        c.titulo_categoria,
        p.texto_pregunta,
        o.texto_opcion
    FROM 
        categorias_chatbot c
    JOIN 
        categoria_pregunta_chat_intermed cp ON c.id_categoria = cp.id_categoria
    JOIN 
        preguntas_chatbot p ON cp.id_pregunta = p.id_pregunta
    JOIN 
        preguntas_opciones_chatbot po ON p.id_pregunta = po.id_pregunta
    JOIN 
        opciones_chatbot o ON po.id_opcion = o.id_opcion
    WHERE 
        c.seccion = 'sociosanitario'
    ORDER BY 
        c.id_categoria, p.id_pregunta, o.id_opcion;
"""


df = pd.read_sql_query(query_usuarios, conn)
df

# OTRO

In [ ]:
import json

# Agrupar opciones por pregunta y categoría
grouped = df.groupby(["id_categoria", "titulo_categoria", "texto_pregunta"])["texto_opcion"].apply(list).reset_index()

# Crear el JSON dinámico
json_data = {}
for _, row in grouped.iterrows():
    id_categoria = row["id_categoria"]
    titulo = row["titulo_categoria"]
    pregunta = row["texto_pregunta"]
    opciones = row["texto_opcion"]
    
    if id_categoria not in json_data:
        json_data[id_categoria] = {"titulo": titulo, "preguntas": {}}
    
    json_data[id_categoria]["preguntas"][pregunta] = opciones

# Convertir a JSON
json_resultado = json.dumps(json_data, indent=4, ensure_ascii=False)

print(json_resultado)

In [12]:
import json

# Agrupar opciones por pregunta y categoría
grouped = df.groupby(["id_categoria", "titulo_categoria", "texto_pregunta"])["texto_opcion"].apply(list).reset_index()

# Crear el JSON dinámico
json_data = {}
for _, row in grouped.iterrows():
    id_categoria = row["id_categoria"]
    titulo = row["titulo_categoria"]
    pregunta = row["texto_pregunta"]
    opciones = row["texto_opcion"]
    
    if id_categoria not in json_data:
        json_data[id_categoria] = {"titulo": titulo, "preguntas": {}}
    
    json_data[id_categoria]["preguntas"][pregunta] = opciones

# Convertir a JSON
json_resultado = json.dumps(json_data, indent=4, ensure_ascii=False)

print(json_resultado)


{
    "1.1": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Cuándo te diagnosticaron?": [
                "Hace menos de 6 meses",
                "Entre 6 meses y un año",
                "Hace más de un año"
            ],
            "¿Estás en tratamiento TAR?": [
                "Sí",
                "No",
                "No estoy seguro"
            ],
            "¿Tienes acceso a un médico?": [
                "Sí",
                "No"
            ]
        }
    },
    "1.2": {
        "titulo": "Creo que me he expuesto al virus",
        "preguntas": {
            "¿Cuándo ocurrió la posible infección?": [
                "Últimas 72h",
                "Hace más de 72h"
            ]
        }
    },
    "2.1": {
        "titulo": "Personal sanitario",
        "preguntas": {
            "¿que necesitas?": [
                "Manejo clínico de pacientes con VIH",
                "Protocolo PEP"
            ]
        }
    },
    "2.2": {
        "titulo

In [12]:
import pandas as pd

# Tabla de categorías
categorias = pd.DataFrame({
    "id_categoria": [1, 2, 3],
    "titulo_categoria": ["Tengo VIH", "Creo que me he expuesto al virus", "Personal sanitario"]
})

# Tabla de preguntas
preguntas = pd.DataFrame({
    "id_pregunta": [1, 2, 3, 4, 5],
    "id_categoria": [1, 1, 1, 2, 3],
    "texto_pregunta": [
        "¿Cuándo te diagnosticaron?",
        "¿Estás en tratamiento TAR?",
        "¿Tienes acceso a un médico?",
        "¿Cuándo ocurrió la posible infección?",
        "¿Qué necesitas?"
    ]
})

# Tabla de opciones
opciones = pd.DataFrame({
    "id_opcion": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    "id_pregunta": [1, 1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 5],
    "texto_opcion": [
        "Hace menos de 6 meses",
        "Entre 6 meses y un año",
        "Hace más de un año",
        "Sí",
        "No",
        "No estoy seguro",
        "Sí",
        "No",
        "Últimas 72h",
        "Hace más de 72h",
        "Manejo clínico de pacientes con VIH",
        "Protocolo PEP"
    ]
})



In [13]:

# 1. Unir preguntas con categorías
preguntas_categorias = pd.merge(
    preguntas,
    categorias,
    on="id_categoria",
    how="inner"
)

# 2. Unir con opciones
final_table = pd.merge(
    opciones,
    preguntas_categorias,
    on="id_pregunta",
    how="inner"
)

In [16]:
# Lista de columnas que deben ir primero
columns_first = ["id_categoria", "id_pregunta", "id_opcion"]

# Añade las columnas restantes
columns_rest = [col for col in final_table.columns if col not in columns_first]

# Reordena las columnas
final_table = final_table[columns_first + columns_rest]

print(final_table)


    id_categoria  id_pregunta  id_opcion                         texto_opcion  \
0              1            1          1                Hace menos de 6 meses   
1              1            1          2               Entre 6 meses y un año   
2              1            1          3                   Hace más de un año   
3              1            2          4                                   Sí   
4              1            2          5                                   No   
5              1            2          6                      No estoy seguro   
6              1            3          7                                   Sí   
7              1            3          8                                   No   
8              2            4          9                          Últimas 72h   
9              2            4         10                      Hace más de 72h   
10             3            5         11  Manejo clínico de pacientes con VIH   
11             3            

In [17]:
final_table

,id_categoria,id_pregunta,id_opcion,texto_opcion,texto_pregunta,titulo_categoria
0,1,1,1,Hace menos de 6 meses,¿Cuándo te diagnosticaron?,Tengo VIH
1,1,1,2,Entre 6 meses y un año,¿Cuándo te diagnosticaron?,Tengo VIH
2,1,1,3,Hace más de un año,¿Cuándo te diagnosticaron?,Tengo VIH
3,1,2,4,Sí,¿Estás en tratamiento TAR?,Tengo VIH
4,1,2,5,No,¿Estás en tratamiento TAR?,Tengo VIH
5,1,2,6,No estoy seguro,¿Estás en tratamiento TAR?,Tengo VIH
6,1,3,7,Sí,¿Tienes acceso a un médico?,Tengo VIH
7,1,3,8,No,¿Tienes acceso a un médico?,Tengo VIH
8,2,4,9,Últimas 72h,¿Cuándo ocurrió la posible infección?,Creo que me he expuesto al virus
9,2,4,10,Hace más de 72h,¿Cuándo ocurrió la posible infección?,Creo que me he expuesto al virus


In [15]:
import json

# Agrupar opciones por pregunta y categoría
grouped = df.groupby(["id_categoria", "titulo_categoria", "texto_pregunta"])["texto_opcion"].apply(list).reset_index()

# Crear el JSON dinámico
json_data = {}
for _, row in grouped.iterrows():
    id_categoria = row["id_categoria"]
    titulo = row["titulo_categoria"]
    pregunta = row["texto_pregunta"]
    opciones = row["texto_opcion"]
    
    if id_categoria not in json_data:
        json_data[id_categoria] = {"titulo": titulo, "preguntas": {}}
    
    json_data[id_categoria]["preguntas"][pregunta] = opciones

# Convertir a JSON
json_resultado = json.dumps(json_data, indent=4, ensure_ascii=False)

print(json_resultado)


{
    "1.1.1": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Cuándo te diagnosticaron?": [
                "Hace menos de 6 meses"
            ]
        }
    },
    "1.1.2": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Cuándo te diagnosticaron?": [
                "Entre 6 meses y un año"
            ]
        }
    },
    "1.1.3": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Cuándo te diagnosticaron?": [
                "Hace más de un año"
            ]
        }
    },
    "1.1.4": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Estás en tratamiento TAR?": [
                "Sí"
            ]
        }
    },
    "1.1.5": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Estás en tratamiento TAR?": [
                "No"
            ]
        }
    },
    "1.1.6": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Estás en tratamiento TAR?": [
     

## probar base de datos a json

In [ ]:


except psycopg2.OperationalError as e:
    raise HTTPException(status_code=400, detail="Error de conexión: " + str(e))
except Exception as error:
    raise HTTPException(status_code=400, detail="Error desconocido: " + str(e))
    
# Escribe la consulta SQL
query = "SELECT * FROM sociosanitarios_data"

# Usa pandas para ejecutar la consulta y convertirla en un DataFrame
df = pd.read_sql_query(query, connection)

# Agrupar opciones por pregunta y categoría
grouped = df.groupby(["id_categoria", "titulo_categoria", "texto_pregunta"])["texto_opcion"].apply(list).reset_index()

# Crear el JSON dinámico
json_data = {}
for _, row in grouped.iterrows():
    id_categoria = row["id_categoria"]
    titulo = row["titulo_categoria"]
    pregunta = row["texto_pregunta"]
    opciones = row["texto_opcion"]
    
    if id_categoria not in json_data:
        json_data[id_categoria] = {"titulo": titulo, "preguntas": {}}
    
    json_data[id_categoria]["preguntas"][pregunta] = opciones

# Convertir a JSON
json_resultado = json.dumps(json_data, indent=4, ensure_ascii=False)

print(json_resultado)


# ESTO ES LO QUE HAY QUE METER***

## UTILS

In [ ]:
import google.generativeai as genai
def formateo_incusivo(prompt):
    
    try:
        prompt_historia = f"""
    Eres un experto en genero neutro y con el pronombres inclusivos como elle, investiga bien como funciona. Analiza el sigiente texto como si estuviese hablanco contigo y formateamelo: {prompt}

    Si crees que debes modificar algo para introduir el pronombre elle y conjugar los adjetivos a neutro, hazlo. Sino devuelveme exactamente el mismo texto.
    No hables de nada, simplemente formatealo y sino lo devuelves como está.
    """
        model = genai.GenerativeModel("gemini-1.5-flash")
        response = model.generate_content(prompt_historia)
        if not response or not hasattr(response, 'text'):
            raise ValueError("Respuesta vacía o no válida del modelo.")
        return response.text
    except Exception as e:
        return f"Error al generar respuesta para historia: {str(e)}"


## MAIN

In [ ]:
def generar_respuesta(prompt):
    try:
        def call_model():
            model = genai.GenerativeModel("gemini-1.5-flash")
            print("*"*50)
            print("prompt_basico:",prompt_basico)
            print("*"*50)
            print("prompt:",prompt)
            print("*"*50)
            prompt_total = prompt_basico + prompt
            print("*"*50)
            print("prompt_total:",prompt_total)
            print("*"*50)
            return model.generate_content(prompt_total)

        # Usar un executor para manejar el tiempo límite
        with concurrent.futures.ThreadPoolExecutor() as executor:
            future = executor.submit(call_model)
            response = future.result(timeout=30)  # Tiempo límite de 30 segundos

        # Verificar si la respuesta es válida
        if not response or not hasattr(response, 'text'):
            raise ValueError("Respuesta vacía o no válida del modelo.")
        
        respuesta_final = formateo_incusivo(response.text) 
        print("*"*50)
        print("/"*50)
        print("respuesta_final:",respuesta_final)
        print("/"*50)
        print("*"*50)
        return respuesta_final
    except concurrent.futures.TimeoutError:
        print("El modelo tomó demasiado tiempo en responder.")
        return "Error: El modelo tardó demasiado en responder."
    except Exception as e:
        print(f"Error en generar_respuesta: {e}")
        return f"Error al generar respuesta para historia: {str(e)}"